# 

Giving a shit about statistics

Every time I take a statistics course -- one that's about inference, design, etc. and less about math, algorithms, and such -- I really have a hard time giving a shit. It feels so boring. I make this document in hopes of giving some shits, but I am doubtful. Nonetheless, I try. 

## Julia

In [1]:
using Plots
using Statistics
using LinearAlgebra
using StatsPlots
using StatsBase
using Distributions
using HypothesisTests
using DataFrames
using CSV
using Optim
using GLM
using LaTeXStrings
using CategoricalArrays

┌ Info: Recompiling stale cache file /home/mosiman/.julia/compiled/v1.1/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/mosiman/.julia/compiled/v1.1/StatsPlots/SiylL.ji for StatsPlots [f3b207a7-027a-5e70-b257-86293d7955fd]
└ @ Base loading.jl:1184


## The Data

### London bikeshare data

from kaggle! https://www.kaggle.com/hmavrodiev/london-bike-sharing-dataset/data

```
"timestamp" - timestamp field for grouping the data
"cnt" - the count of a new bike shares
"t1" - real temperature in C
"t2" - temperature in C "feels like"
"hum" - humidity in percentage
"wind_speed" - wind speed in km/h
"weather_code" - category of the weather
"is_holiday" - boolean field - 1 holiday / 0 non holiday
"is_weekend" - boolean field - 1 if the day is weekend
"season" - category field meteorological seasons: 0-spring ; 1-summer; 2-fall; 3-winter.

"weathe_code" category description:
1 = Clear ; mostly clear but have some values with haze/fog/patches of fog/ fog in vicinity
2 = scattered clouds / few clouds
3 = Broken clouds
4 = Cloudy
7 = Rain/ light Rain shower/ Light rain
10 = rain with thunderstorm
26 = snowfall
94 = Freezing Fog
```

In [2]:
bikedata = CSV.read("london_merged.csv")
head(bikedata)

┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[2]:2
└ @ Core In[2]:2


,timestamp,cnt,t1,t2,hum,wind_speed,weather_code
,String,Int64,Float64,Float64,Float64,Float64,Float64
1,2015-01-04 00:00:00,182,3.0,2.0,93.0,6.0,3.0
2,2015-01-04 01:00:00,138,3.0,2.5,93.0,5.0,1.0
3,2015-01-04 02:00:00,134,2.5,2.5,96.5,0.0,1.0
4,2015-01-04 03:00:00,72,2.0,2.0,100.0,0.0,1.0
5,2015-01-04 04:00:00,47,2.0,0.0,93.0,6.5,1.0
6,2015-01-04 05:00:00,46,2.0,2.0,93.0,4.0,1.0


In [3]:
# make weather_code categorical
bikedata.weather_code = categorical(bikedata.weather_code)

┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = catvaluetype at array.jl:614 [inlined]
└ @ Core /home/mosiman/.julia/packages/CategoricalArrays/xjesC/src/array.jl:614


17414-element CategoricalArray{Float64,1,UInt32}:
 3.0
 1.0
 1.0
 1.0
 1.0
 1.0
 4.0
 4.0
 4.0
 3.0
 3.0
 3.0
 4.0
 ⋮  
 3.0
 4.0
 3.0
 4.0
 4.0
 3.0
 2.0
 3.0
 4.0
 4.0
 4.0
 2.0

┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = show(::IOContext{Base.GenericIOBuffer{Array{UInt8,1}}}, ::CategoricalValue{Float64,UInt32}) at value.jl:104
└ @ CategoricalArrays /home/mosiman/.julia/packages/CategoricalArrays/xjesC/src/value.jl:104
┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = show(::IOContext{Base.GenericIOBuffer{Array{UInt8,1}}}, ::CategoricalValue{Float64,UInt32}) at value.jl:104
└ @ CategoricalArrays /home/mosiman/.julia/packages/CategoricalArrays/xjesC/src/value.jl:104


In [ ]:
# realized I should probably have done this before all the analysis, so adding it to the top here
# make timestamp a datetime

using Dates

dateformat = DateFormat("Y-m-d H:M:S")

bikedata.timestamp = DateTime.(bikedata.timestamp, dateformat)

In [ ]:
BIKENUMSAMPLE = 250
bike = bikedata[sample(axes(bikedata, 1), BIKENUMSAMPLE; replace = false, ordered = true), :]

In [ ]:
bike_filter = filter(x -> x.cnt > 0, bike)
scatter(bike_filter.t1, bike_filter.cnt)

## Linear Regression

### The model

The model we use to describe linearly correlated data is 

$$
y = \beta_0 x + \beta_1 + \epsilon
$$

Which is quite reminiscent of our highschool $y = mx + b$ days, except we have a random term $\epsilon$ where $\epsilon \sim N(0,\sigma^2)$. 

We assume, for simplicity, that our $x$ is 'controllable', that is, not random. This gives us our first important result, which is that our $y$ are normally distributed with the same variance as our random terms. 

$$
y_i \sim N(\beta_0 + \beta_1 x_i, \sigma^2)
$$

### Parameter estimation

How do we find out what the parameters are? We have the following two approaches:

#### Least Squares Estimates

The least squares estimates are exactly what they sound like. They are the values that minimize the squared distance between the observed response and a fitted line. I.e.,

$$
\arg \min \sum_{i=1}^n \left[ y_i - (\hat\beta_0 + \hat\beta_1 x_i) \right]^2
$$

Where the hats represent the fact that these are only _estimates_ for the true parameter values.

The minimum values can be found analytically in this simple case. Using some partial derivatives and finding extreme values (also, checking the second derivatives), it is not too difficult to come to the conclusion:

$$
\hat\beta_0 = \bar y - \hat\beta_1 \bar x
$$

$$
\hat\beta_1 = \frac{\sum x_i y_i - n \bar x \bar y}{\sum x_i^2 - n\bar x^2}
$$

We can use this to fit a model for temperature vs number of bikes ridden

In [ ]:
n = size(bike,1)
x̄ = mean(bike.t1)
ȳ = mean(bike.cnt)
β₁ = (sum([ b.t1 * b.cnt for b in eachrow(bike)]) - n*x̄*ȳ ) / ( sum([b.t1^2 for b in eachrow(bike)]) - n*x̄^2)
β₀ = ȳ - β₁ * x̄

(β₀, β₁)

In [ ]:
scatter(bike.t1, bike.cnt, xlab="temperature", ylab="count", label="Bike count")
plot!( x -> β₀ + β₁*x, 0:30, linewidth=3, label="Fitted regression")

The least squares estimates also provide us with some properties of residuals. Note that because the least squares estimates satisfy 

$$
(\hat\beta_0, \hat\beta_1) = \arg \min \sum_{i=1}^n (\hat y_i - y_i)^2 
$$

We have the following conditions as a consequence of $\hat\beta_0$ and $\hat\beta_1$ being critical values

$$
\begin{aligned}
    0 & = \sum_{i=1}^n (y_i - \hat\beta_0 - \hat\beta_1x_i) \\
        & = \sum_{i=1}^n r_i = 0 \\
\end{aligned}
$$

and

$$
\begin{aligned}
    0 & = \sum_{i=1}^n (y_i - \hat\beta_0 - \hat\beta_1x_i) (-x_i)\\
        & = \sum_{i=1}^n (y_i - \hat y_i)(-x_i) \\
        & = \sum_{i=1}^n \hat y_i x_i - y_i x_i \\
        & = \sum_{i=1}^n (\hat y_i - y_i)x_i \\
        & = \sum_{i=1}^n r_i x_i
\end{aligned}
$$

Using the fact that these sums involving residuals go to $0$ using least squares estimates, we can conclude one more: $\sum_{i=1}^n r_i \hat y_i = 0$. 

$$
\begin{aligned}
    \sum r_i \hat y_i = 0 & \iff \sum r_i x_i + \sum r_i \hat y_i = 0\\
                                            & \iff \sum r_i (\hat y_i + x_i) = 0 \\
                                            & \iff ( \hat\beta_0 \sum r_i) + (\hat\beta_1 \sum r_i x_i ) + (\sum r_i x_i)  = 0\\
                                            & \iff 0 + 0 + 0 = 0
\end{aligned}
$$

#### Maximum likelihood estimates

The maximum likelihood estimates come from the following:

$$
\arg \max L(\theta | x_1, \cdots, x_n)
$$

where  $L(\theta | x_1, \cdots, x_n) = \Pi_{i=1}^n f(x_i | \theta)$. 

Note that in our case, we would like to estimate the $\beta_0, \beta_1$ parameters. The probability function for $y$ is normal, in fact, $y_i \sim N(\beta_0 + \beta_1 x_i, \sigma^2)$. Therefore, we can solve for the maximum likelihood of three parameters: $\beta_0, \beta_1, \sigma$. 

Use the log likelihood, as $\log$ is a strictly increasing function and therefore preserves our critical values.

The values we find are

- $\hat\beta_0 = \bar y - \beta_1 \bar x$
- $\hat\beta_1 = \frac{\sum x_i y_i - n \bar x \bar y}{\sum x_i^2 - n\bar x^2}$
- $\hat\sigma^2 = \frac 1n \sum_{i=1}^n r_i^2$

Note: that maximum likelihood estimate gives a __biased__ estimator for $\sigma^2$, but our estimates for $\hat\beta_0$ and $\hat\beta_1$ are identical to the least squares estimates.

#### Numerical estimation

We can estimate numerically these quantities with some sort of numerical solver.  We'll use Julia's `Optim.jl` package. 

this was a bust. Redo when possible!

In [ ]:
# the pdf of the response, since the response is normal
# θ = [β₀, β₁, σ]
f(x, θ) = pdf(Normal(θ[1] + θ[2]*x, θ[3]^2), x)
L(θ) = prod(map( x -> f(x, θ), bike.t1))
l(θ) = sum(map(x -> log(f(x, θ)), bike.t1) )

optimize(x -> l(x), [β₀, β₁, 100000.])

### Hypothesis testing

Hypothesis testing is to inquire about the validity of a statement about a parameter. I have written a little on the logic behind this. The gist is, we need distributional results on our parameters. This is not a crazy question: our estimates are a function of the response, which we have concluded are random variables.

#### $t$ statistic and tests

A $t$ statistic can be thought of as a $z$ statistic, but having replaced the unknown quantity $\sigma^2$ with the estimate $S^2$.  That is

$$
t = \frac{\bar X - \mu }{S / \sqrt{n} } \sim t_{n-1}
$$

The $t$ statistic follows a $t$ distribution with $n-1$ degrees of freedom. 

By assuming the null hypothesis, the $t$ statistic can be computed. The probability distribution of the $t$ statistic allows us to conclude if the probability of observing something as unlikely as our statistic is probably, thus garning evidence for the rejection of lack thereof of the hypothesis.


#### $f$ statistic and tests

The usual characterization for an $F$ statistic is that a quantity $F$ has an $F$ distribution if it assumes the form 

$$
F = \frac{U_1 / d_1}{U_2 / d_2} \sim F_{d_1, d_2}
$$

I.e., if $F$ can be expressed as the ratio of two quantities $U_1$ and $U_2$ who are $\chi^2$ distributed with $d_1$ and $d_2$ degrees of freedom, respectively, then $F$ is $F_{d_1, d_2}$-distributed.

Below, we see that SSE and SSR are both $\chi^2$ distributed. In fact, they have degrees of freedom $1, n-2$ respectively. Thus an $F$ test arising naturally from the data has the form 

$$
F = \frac{\text{SSR}/1}{\text{SSE}/(n-2)} = \frac{\text{MSR}}{\text{MSE}} \sim F_{1, n-2}
$$

SSR encodes the descrepency between a basic bitch model $y = b$ and a big brain model $y = mx + b$. If SSR is large, i.e., the ratio $\frac{\text{MSR}}{\text{MSE}}$ is large, then we likely have data dependent on $x_i$'s. 

This is more or less equivalent to testing if $\beta_1 = 0$. In fact, $\text{MSR} / \text{MSE}$ is $F$-distributed only under the null hypothesis that $\beta_1 = 0$. 

In [ ]:
plot(x -> pdf(FDist(1, 3), x), 0:0.003:2.5, label=L"F_{1, 3}", linewidth=3, title="F distribution with 1,3 d.f.")

### ANOVA

I don't understand which it's such a big deal.. and I haven't heard this term ever used in a natural way. Whatever. Supposedly, any decent exposition on this subject starts with the topic of decomposing variance. 

Recall the variance 

$$
\text{var}(y) = \sum (y_i - \mu)^2
$$

I.e., the sum squared distance from the mean. We don't have the mean, so consider instead

$$
\sum (y_i - \bar y )^2
$$

A decomposition:

$$
\sum (y_i - \bar y)^2 = \left[ \sum (y_i - \hat y_i)^2 \right] + \left[ \sum (\hat y_i - \bar y)^2 \right]
$$

Or:

$$
\text{SST} = \text{SSE} + \text{SSR}
$$

Consider the following proof by picture.


In [ ]:
xs = [ 1 , 3 , 4 ,10 ,11 ,15 ,16 ,17]
ys = [21.55219363265408  ,  8.70482469035426  , 27.451668121513542 , 43.66593098495369  , 46.84274178061619  , 77.08568993110022  , 99.3367189562757   , 90.41233511277945  ]

df = DataFrame(X = xs, Y = ys)
lmfit = lm(@formula(Y ~ X), df)

β₀, β₁= coef(lmfit)

ȳ = mean(ys)

i = 7
ŷᵢ_pos = [xs[7], β₀ + β₁*xs[7]]
ȳ_pos = [xs[7], ȳ]
yᵢ_pos = [xs[7], ys[7]]

scatter(xs, ys, label= "", legend=:left, title="Visual proof of total variance decomposition")
plot!(x -> β₀ + β₁*x, 0:20, label= L"\hat y_i")
plot!(x -> ȳ, 0:20, label=L"\bar y")
#plot!(yᵢ_pos, ŷᵢ_pos)
plot!([xs[7], xs[7]], [β₀ + β₁*xs[7], ys[7]], label=L"\hat y_i - y_i", linewidth=2)
plot!([xs[7], xs[7]], [β₀ + β₁*xs[7], ȳ], label=L"\hat y_i - \bar y", linewidth=2)
plot!([xs[6]+0.5, xs[6]+0.5], [ȳ, ys[7]], label=L"y_i - \bar y", linewidth=2)

So, pointwise, the distance between $y_i$ and the mean $\bar y$ is clearly the distance from $y_i$ to $\hat y_i$ plus the distance from $\hat y_i$ to $\bar y$. 

## To $\mathbb{R}^2$ and beyond 

Let's model this shit

In [ ]:
cornerplot(hcat(bike.cnt, bike.t1, bike.t1, bike.hum, bike.wind_speed, bike.weather_code))

In [ ]:
@df bike corrplot(cols(2:6))

Looks like `t1`, `t2`, `hum`, and `wind_speed` are pretty skewed. I wouldn't particularly expect count to be 'normal', more like poisson so thats good. 

Count is somewhat linearly correlated with temperature, negatively correlated with humidity. 


In [ ]:
# try a model with t1, t2, hum, wind_speed

bikeBigModel = lm(@formula(cnt ~ t1 + t2 + hum + wind_speed + weather_code), bike)

If I include the entire dataset, I get more significant results. I'll try that.

In [ ]:
bikeBigModel = lm(@formula(cnt ~ t1 + t2 + hum + wind_speed + weather_code), bikedata)

So temperature, humidity, wind speed, are significant, and certain weather codes (2 (cloudy), 3 (broken clouds), maybe even 4 ) are more significant compared to weather code 1

In [ ]:
function vif(dat)
    # VIF_j = 1 / (1 - Rⱼ²)
    # for each variable $xⱼ$, regress $xⱼ$ on all other variables 
    # assume dat's variables can be accessed by indexing, i.e., dat[j]
    # also assume it has a header, so basically dat should be a dataframe.
    vars = names(dat)
    r2_vals = []
    for j in 1:size(dat)[2]
        # regress x_j on all others
        fit_j = lm(Matrix(dat[:,1:end .!= j]), dat[j])
        append!(r2_vals, r2(fit_j))
    end
    vif_vals = map(x -> 1 / (1-x), r2_vals)
    return vif_vals
end



        

In [ ]:
vif(bike[[:cnt, :t1, :t2, :hum, :wind_speed]])

We see that `t1` and `t2` are very collinear, so let's just take `t1`. 

In [ ]:
bikeModel = lm(@formula(cnt ~ t1 + hum + wind_speed + weather_code), bikedata)

Let's get some diagnostic plots going

nevermind, wow lol it's so easy in R but there are no Julia functions for easy diagnostic plots from a dataframe / linear model rip

From above it looked like humidity and wind speed were skewed, so we can try a log or power transformation.

In [ ]:
# box cox 
using BoxCoxTrans

an example of optimizing for the appropriate $\lambda$ for the power transform:

In [ ]:
BoxCoxTrans.lambda(bike.wind_speed, interval=(-5,5))

We have a problem, because the box cox transform requires that $y_i > 0$ strictly, some of the wind speed is $0$ for some days.

Out of $17414$ days there are 68 days with "0" wind speed. I'm not sure if this is N/A or if there was actually just no wind.

Some googling suggests its not uncommon for calm days (less than 1kph) to be marked as 0 wind speed. In that case, we should just add a small epsilon to any 0 wind speed days. 





In [ ]:
wind_speed_plusEpsilon = map(x -> iszero(x) ? x += 0.1 : x, bikedata.wind_speed)

In [ ]:
λhum = BoxCoxTrans.lambda(bikedata.hum, interval=(-5,5)).value
λwind_speed = BoxCoxTrans.lambda(wind_speed_plusEpsilon, interval=(-5,5)).value

Let's transform them and make a new dataframe with the appropriate data

In [ ]:
bikedata_pt = copy(bikedata)

bikedata_pt.wind_speed = BoxCoxTrans.transform(wind_speed_plusEpsilon, λwind_speed)
bikedata_pt.hum = BoxCoxTrans.transform(bikedata.hum, λhum)

In [ ]:
bikeModelPT = lm(@formula(cnt ~ t1 + hum + wind_speed + weather_code), bikedata_pt)

Plotting all the bike data takes way too long on my potato laptop. I tried using a 500 element subset, but it was too small and missed out on many things. I will try a 2500 element subset just for plotting. 

In [ ]:
bike_pt = bikedata_pt[sample(axes(bikedata_pt, 1), 2500; replace = false, ordered = true), :]

In [ ]:
@df bike_pt corrplot(cols(2:6))

Now, are any of the weather codes significant? We only know that weather codes 2, 3, and 4 are significant but only compared to weather code 1. We can use an F test. 

In [ ]:
fitWithoutWeatherCode = lm(@formula(cnt ~ t1 + hum + wind_speed), bikedata_pt)
fitWithWeatherCode = lm(@formula(cnt ~ t1 + hum + wind_speed + weather_code), bikedata_pt)

ftest(fitWithWeatherCode.model, fitWithoutWeatherCode.model)

It is significant, by a lot, so let's try this as a final model.

We can try plotting the true data versus our model's predicted values

We'll take a subset of the data, but first we should convert the timestamp column to an actual DateTime format (instead of a plain string) so we can query it easily.

In [ ]:

#plotregion = bikedata[ 
 #   (Dates.month(bikedata_pt.timestamp) == 8) .& (Dates.year(bikedata_pt.timestamp) == 2016)
#]

#plotregion = map(x -> Dates.month(x) == 8 && Dates.year(x) == 2016 ? 1 : 0, 
#    bikedata_pt.timestamp)
plotregion = filter(x-> Dates.week(x) == 33 && Dates.year(x) == 2016,
    bikedata_pt.timestamp)

df_subset = filter(row -> row.timestamp in plotregion, bikedata_pt)

# predicted values in plotregion

predicted = predict(bikeModelPT, df_subset )

plot(plotregion, df_subset.cnt)
plot!(plotregion, predicted)

wow, that's pretty shit lol

looks like the predicted values peak at noon-ish, and I think that's because that's when temperature would be highest. 

In the actual data, we have a sort of bimodal distribution over the day, which I think might map to going to work and coming back from work. There's a clear lull in the middle of the day. 

What I had in mind for my model, was that it could be used to take a weekly forecast and appropriately schedule workers for that day / week. For example, I know in Toronto the bikeshare system requires workers to load balance the access points, so there's always some bikes at each (and space at each, to park). By predicting usage, we may also be able to predict how many people we'll need to shuffle around bicycles. 

Let's use the same model, but change the dataset a bit. Instead of hourly counts, let's take daily counts. We'll then use variables `t_hi` and `t_lo` to represent the high and lows of that day. Let's also take average wind speed and the mode of the weather code for that day. 

I'll load some more packages to help with the data wrangling

In [ ]:
using Query

In [ ]:
bikedaily = bikedata |> 
    @groupby(x -> Date(x.timestamp))  |> 
    @map( {Date = key(_), 
                    cnt = mean(_.cnt), 
                    thi = maximum(_.t1), 
                    tlo = minimum(_.t1),
                    humhi = maximum(_.hum), 
                    humlo = minimum(_.hum),
                    wind_speed = mean(_.wind_speed),
                    weather_code=mode(_.weather_code)}) |> 
    DataFrame

In [ ]:
# cornerplot
@df bikedaily corrplot(cols(2:7))

Interesting to note, is that `cnt` looks like a different poisson distribution. 

In [ ]:
dailyfit = lm(@formula(cnt ~ thi + tlo + humhi + humlo + wind_speed + weather_code), bikedaily)
dailyfit

ah oops, I want the t-test to compare levels with weather code 1 as the 'reference' level. We reorder the categorical variable

In [ ]:
levels!(bikedaily.weather_code, [1.0, 2.0, 3.0, 4.0, 7.0, 26.0])

dailyfit = lm(@formula(cnt ~ thi + tlo + humhi + humlo + wind_speed + weather_code), bikedaily)
dailyfit

I know `thi` and `tlo` are collinear, so are `humhi` and `humlo`. We can see this using the variance inflation factor.

In [ ]:
vif(bikedaily[[:thi, :tlo, :humhi, :humlo, :wind_speed]])

Interestingly, not as collinear as I thought.

On the other hand, it looks like the weather codes are not as significant as they were before. It's worth testing to see if weather codes are significant at all. 

In [ ]:
fitWithoutWeatherCode = lm(@formula(cnt ~ thi + tlo + humhi + humlo + wind_speed), bikedaily)
fitWithWeatherCode = lm(@formula(cnt ~ thi + tlo + humhi + humlo + wind_speed + weather_code), bikedaily)

ftest(fitWithWeatherCode.model, fitWithoutWeatherCode.model)

Weather codes are significant. So I guess we can conclude that the weather conditions do not really significantly affect ridership when compared to the reference level (clear skies), unless it is weather code 7 (rainy!).

Because we took the mode of the weather codes for the day, it looks like we're missing out on some weather codes but that's fine I suppose.

Let's try comparing our model like above.

In [ ]:

plotregion = filter(x-> Dates.month(x) in [6,7,8,9] && Dates.year(x) == 2016,
    bikedaily.Date)

df_subset = filter(row -> row.Date in plotregion, bikedaily)

# predicted values in plotregion

predicted = predict(dailyfit, df_subset )

plot(plotregion, df_subset.cnt, lw=2, label="Historical")
plot!(plotregion, predicted, lw=2, label="Model")

This seems to be doing a lot better, although it feels suspiciously like overfitting, since all the movements are the very similar, only differing in magnitude. 

It would be nice to plot some confidence bands. Gotta pull out some math though, I think, since the packages won't do it for me

Recall some common properties

- $\hat \beta \sim MVN(\beta, \sigma^2(X^TX)^{-1})$

    Since $\hat \beta = Hy = (X^TX)^{-1}X^T(X\beta + \epsilon)$, everything is deterministic other than epsilon, which has distribution $N(0,\sigma^2)$.  So we know that $\hat \beta$ is normal. We just need its mean and variance. 
    
    The mean comes easily, since $E(\epsilon) = 0$ and the $X$s cancel out. We end up with $E(\hat \beta) = \beta$. Unbiased, nice.
    
    The variance is more algebra as well. 
    
    $$
    \begin{aligned}
        \text{Var}(\hat \beta) & = \text{Var}((X^TX)^{-1}X^T(X\beta + \epsilon)) \\
                                               & = \text{Var}(\beta + (X^TX)^{-1}X^T\epsilon) \\
                                               & = 0 + \text{Var}((X^TX)^{-1}X^T\epsilon) \\
                                               & = (X^TX)^{-1}X^T \text{Var}(\epsilon) ( (X^TX)^{-1}X^T)^T \\
                                               & = \sigma^2 (X^TX)^{-1}
     \end{aligned}
     $$
     
- Prediction intervals 

    Consider the prediction error 
    
    $$
    y_p - \hat \mu_p
    $$
    
    Where $y_p$ is the true value at $x_p$ and $\hat \mu_p$ is the predicted value at $x_p$. 
    
    We can construct a distribution on the prediction error. Note that $y_p - \mu_p = \mu_p + \epsilon - \hat \mu_p$. Therefore we know it's normally distributed. The mean is clearly zero, since $\hat \mu_p = x_p^T \beta$ is unbiased. As for the variance,
    
    $$
    \begin{aligned}
        \text{Var}(y_p - \hat \mu_p) & = 0 + \sigma^2 + \text{Var}(\hat \mu_p) \\
                                                          & = \sigma^2 + \text{Var}(x_p^T \hat\beta) \\
                                                          & = \sigma^2 + x_p^T \sigma^2 I (X^TX)^{-1} x_p \\
                                                          & = \sigma^2 ( 1 + x_p^T (X^TX)^{-1} x_p )
     \end{aligned}
     $$
     
     So $y_p - \hat \mu_p \sim N(0, \sigma^2(1 + x_p^T (X^TX)^{-1} x_p)$. 
     
     Then we have 
     
     $$
     \frac{(y_p - \hat \mu_p) - 0}{ \text{se}(y_p - \hat\mu_p) } \sim t_{n-p-1}
     $$
     
     where $\text{se}(y_p - \hat\mu_p) = \sqrt{\hat\sigma^2(1 + x_p^T (X^TX)^{-1} x_p)}$
     
     Therefore, a confidence interval for the prediction with confidence level $\alpha$ is 
     
     $$
     \hat\mu_p \pm t_{n-p-1}^{\alpha / 2} \text{se}(y_p - \hat\mu_p)
     $$

In [ ]:
# yeah, I don't think this is the best (numerically), i.e. not taking advantage of matrix properties but whatever


α = 0.05
X = dailyfit.mm.m # design matrix
e = residuals(dailyfit)
S² = e' * e / (length(e) - 11)
tᵅ = ccdf(TDist(length(e) - 11), α/2)

function se_pred(X, σ̂², xₚ)
    return √(σ̂² * (1 + xₚ' * (X'X) * xₚ))
end

In [ ]:
plotregion = filter(x-> Dates.month(x) in [6,7,8,9] && Dates.year(x) == 2016,
    bikedaily.Date)

df_subset = filter(row -> row.Date in plotregion, bikedaily)


dailyfit_subset = lm(@formula(cnt ~ thi + tlo + humhi + humlo + wind_speed + weather_code), df_subset)

# predicted values in plotregion

# Need to get the observations xₚ from df_subset. Problem is xₚ needs our dummy encodings.
# yeah, not too happy with my solution for this but whatever

covar_subset = dailyfit_subset.mm.m
#ha, this is sketch. Doing it cuz this subset doesn't contain all the weather codes I need
covar_subset = hcat(covar_subset, zeros(size(covar_subset[1])))
se_predictions = [se_pred(X,S², xp) for xp in eachrow(covar_subset)]

predicted = predict(dailyfit, df_subset)

plot(plotregion, df_subset.cnt, lw=2, label="Historical")
plot!(plotregion, predicted, lw=2, label="Model")
plot!(plotregion, )